## Retriever and Chain with Langchain

In [1]:
## PDF Loader
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('paper.pdf')
docs = loader.load()
docs

[Document(metadata={'producer': 'Acrobat Distiller 10.1.8 (Windows)', 'creator': 'Springer', 'creationdate': '2020-08-18T14:15:04+05:30', 'author': 'Vijaykumar Patil', 'crossmarkdomains[1]': 'springer.com', 'crossmarkdomains[2]': 'springerlink.com', 'crossmarkdomainexclusive': 'true', 'crossmarkmajorversiondate': '2010-04-23', 'keywords': 'ABO typing,Blood groups,Fingerprint map reading,Machine learning,Deep leaning,Artificial intelligence', 'moddate': '2021-02-20T15:29:06+05:30', 'subject': 'Artificial Intelligence Review, https://doi.org/10.1007/s10462-020-09891-w', 'title': 'An association between fingerprint patterns with blood group and lifestyle based diseases: a review', 'doi': '10.1007/s10462-020-09891-w', 'robots': 'noindex', 'source': 'paper.pdf', 'total_pages': 37, 'page': 0, 'page_label': '1803'}, page_content='Vol.:(0123456789)\nArtificial Intelligence Review (2021) 54:1803–1839\nhttps://doi.org/10.1007/s10462-020-09891-w\n1 3\nAn association between\xa0fingerprint pattern

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
chunk_documents = text_splitter.split_documents(docs)
chunk_documents

[Document(metadata={'producer': 'Acrobat Distiller 10.1.8 (Windows)', 'creator': 'Springer', 'creationdate': '2020-08-18T14:15:04+05:30', 'author': 'Vijaykumar Patil', 'crossmarkdomains[1]': 'springer.com', 'crossmarkdomains[2]': 'springerlink.com', 'crossmarkdomainexclusive': 'true', 'crossmarkmajorversiondate': '2010-04-23', 'keywords': 'ABO typing,Blood groups,Fingerprint map reading,Machine learning,Deep leaning,Artificial intelligence', 'moddate': '2021-02-20T15:29:06+05:30', 'subject': 'Artificial Intelligence Review, https://doi.org/10.1007/s10462-020-09891-w', 'title': 'An association between fingerprint patterns with blood group and lifestyle based diseases: a review', 'doi': '10.1007/s10462-020-09891-w', 'robots': 'noindex', 'source': 'paper.pdf', 'total_pages': 37, 'page': 0, 'page_label': '1803'}, page_content='Vol.:(0123456789)\nArtificial Intelligence Review (2021) 54:1803–1839\nhttps://doi.org/10.1007/s10462-020-09891-w\n1 3\nAn association between\xa0fingerprint pattern

In [3]:
## Vector Embedding and Vector Store using Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(chunk_documents, OllamaEmbeddings())
db

/tmp/ipykernel_32178/2625579860.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = FAISS.from_documents(chunk_documents, OllamaEmbeddings())


In [4]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>
                                          
Question:{input}

""")

In [5]:
from langchain_community.llms import Ollama
## Ollama llama2
llm = Ollama(model="llama2")
llm

/tmp/ipykernel_32178/2724951080.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


Ollama()

In [6]:
## Chain 
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context.\nThink step by step before providing a detailed answer.\nI will tip you $1000 if the user finds the answer helpful.\n<context>\n{context}\n</context>\n\nQuestion:{input}\n\n'), additional_kwargs={})])
| Ollama()
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [7]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x793ebcd39220>, search_kwargs={})

In [8]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [9]:
response = retrieval_chain.invoke({"input":"Fingerprint map reading"})
response['answer']

'Thank you for providing the context! Based on the information provided, I can answer your question as follows:\n\nFingerprint map reading is a technique used to determine the blood type of an individual based on the patterns found in their fingerprints. The process involves analyzing the ridges and valleys in the fingerprints to determine if there are any specific patterns that indicate a particular blood type.\n\nThe study you mentioned in the context, conducted by Pimenta et al. (2012), showed that dermatoglyphics (the study of fingerprints) can be used as an indicative tool for predicting the chance of developing diabetes in the future. However, the study did not investigate the relationship between fingerprint patterns and blood type.\n\nTherefore, based on the information provided in the context, it is not possible to determine the blood type of an individual based solely on their fingerprint map reading. The traditional method of determining blood type through agglutination test